In [1]:
image_paths = {
    '2018': 'dataset/2018/m_3912112_sw_10_060_20180718.tif',
    '2020': 'dataset/2020/m_3912112_sw_10_060_20200709.tif',
    '2022': 'dataset/2022/m_3912112_sw_10_060_20220714.tif'
}

In [5]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

ref_path = image_paths["2018"]
to_align_path = image_paths["2022"]
aligned_output_path = "dataset/2022/2022_naip_aligned.tif"

# Open the reference raster
with rasterio.open(ref_path) as ref:
    ref_transform = ref.transform
    ref_crs = ref.crs
    ref_width = ref.width
    ref_height = ref.height

# Open the raster to align
with rasterio.open(to_align_path) as src:
    transform, width, height = calculate_default_transform(
        src.crs, 
        ref_crs, 
        src.width, 
        src.height, 
        *src.bounds,
        resolution=(ref_transform.a, ref_transform.e)  # match ref pixel size
    )
    profile = src.profile.copy()
    profile.update({
        'crs': ref_crs,
        'transform': ref_transform,
        'width': ref_width,
        'height': ref_height
    })
    
    with rasterio.open(aligned_output_path, 'w', **profile) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=ref_transform,
                dst_crs=ref_crs,
                resampling=Resampling.cubic
            )